# **Proceso de ETL**
---


En este notebook realizaremos la extracción, transformación y carga (ETL) de los conjuntos de datos proporcionados

## **Extraccion**

In [1]:
#Importamos las librerias necesarias para el desarrollo de el ETL
import numpy as np
import pandas as pd
import ast

In [2]:
# Inicializamos una lista vacía para almacenar los diccionarios
rows = []

# Abrimos el archivo "steam_games.json" en modo lectura
with open("steam_games.json") as f:
    # Iteramos sobre cada línea del archivo
    for line in f.readlines():
        # Utilizamos ast.literal_eval para convertir la línea en un diccionario
        rows.append(ast.literal_eval(line))

# Creamos el DataFrame a partir de la lista de diccionarios
df = pd.DataFrame(rows)


## **Transformacion**


Observamos el df y su estructura para descartar las columnas que a simple vista no nos servirán

In [3]:
# Mostramos los primeros 5 registros del DataFrame
df.head(5)

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,NaN,NaN
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   discount_price  225 non-null    float64
 8   reviews_url     32133 non-null  object 
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  bool   
 12  id              32133 non-null  object 
 13  developer       28836 non-null  object 
 14  sentiment       24953 non-null  object 
 15  metascore       2677 non-null   object 
dtypes: bool(1), float64(1), object(14)
memory usage: 3.7+ MB


In [5]:
df_filtrado = df[["genres", "release_date" , "tags", "specs", "price", "sentiment", "metascore"]]

Dsecartamos todos los Nulos y datos que no tienen el formato correcto de Fecha


In [6]:
#Todos los que no tengan el formato de fecha correcto de la columna release_date los descartamos
df_filtrado = df_filtrado[df_filtrado['release_date'].str.contains(r'\d{4}-\d{2}-\d{2}', na=False)]


Todos los juegos que tengan en sus generos Free to Play le signamos el precio de 0

In [7]:
#Funcion para buscar dentro de lista de generos de cada juegos y asignar 0 si es Free to Play
def set_price_to_zero(row):
    genres = row['genres']
    price = row['price']
    if isinstance(genres, list) and 'Free to Play' in genres:
        return 0
    return price

# Utiliza el método apply para aplicar la función a cada fila del DataFrame
df_filtrado['price'] = df_filtrado.apply(set_price_to_zero, axis=1)

Vemos que dentro de los valores unicos de price hay textos, por ende, le asignamos el valor de Nulo a todos ellos y los borramos

In [8]:
# Vemos los valores unicos de price
df_filtrado["price"].unique()

array([4.99, 0, 0.99, 3.99, nan, 10.99, 2.99, 1.59, 1.99, 9.99, 8.99,
       6.99, 7.99, 39.99, 'Free', 19.99, 7.49, 14.99, 12.99, 5.99, 2.49,
       15.99, 1.25, 29.99, 'Free to Play', 24.99, 17.99, 61.99, 3.49,
       18.99, 11.99, 13.99, 'Free Demo', 34.99, 1.49, 32.99, 99.99, 14.95,
       59.99, 69.99, 5.0, 49.99, 13.98, 29.96, 119.99, 79.99, 109.99,
       16.99, 771.71, 'Install Now', 21.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 'Free To Play', 4.29, 64.99,
       54.99, 74.99, 'Install Theme', 0.89, 'Third-party', 0.5,
       'Play Now', 89.99, 299.99, 44.99, 3.0, 15.0, 5.49, 23.99, 49.0,
       20.99, 10.93, 1.39, 'Free HITMAN™ Holiday Pack', 36.99, 4.49, 2.0,
       4.0, 149.99, 234.99, 1.95, 1.5, 199.0, 189.0, 6.66, 27.99, 10.49,
       129.99, 179.0, 26.99, 399.99, 31.99, 399.0, 20.0, 40.0, 3.33,
       22.99, 320.0, 995.0, 27.49, 6.0, 1.29, 499.99, 199.99, 16.06, 4.68,
       202.76, 1.0, 2.3, 0.95, 172.24, 2.97, 10.96, 19.95, 10.0, 30.0,
       2.66, 6.48, 19.29, 

In [9]:
# Todos los valores que sean numeros los transformamos a valores enteros y los que no en Nulos
df_filtrado['price'] = pd.to_numeric(df_filtrado['price'], errors='coerce')

# Elimina las filas con valores NaN en la columna "price"
df_filtrado = df_filtrado.dropna(subset=['price'])

Dentro de los Valores únicos de la columna sentiment hay muchos que nos dice la cantidad de usuarios que dio su opinión, de este datos no se puede sacar ninguna conclusión con respecto al sentimiento .Ya que no hay manera de saber si la opiniones son malas o buenas, por ende, las descartamos.

In [10]:
# Valores unicos de sentiment
df['sentiment'].unique()


array([nan, 'Mostly Positive', 'Mixed', '1 user reviews',
       '3 user reviews', '8 user reviews', 'Very Positive',
       'Overwhelmingly Positive', '6 user reviews', '5 user reviews',
       '2 user reviews', 'Very Negative', 'Positive', 'Mostly Negative',
       '9 user reviews', 'Negative', '4 user reviews', '7 user reviews',
       'Overwhelmingly Negative'], dtype=object)

In [11]:
# Descartamos las filas que contienen la cadena "user reviews" en la columna "ratings"
df_filtrado = df_filtrado[~df_filtrado['sentiment'].str.contains('user reviews', na=True, regex=False)]

Hay valores NA en metascore que pandas no los reconoce como Nulo, por ende los reemplazamos por None y descartamos todos los valores nulos

In [12]:
# Valores unicos de sentiment
df['metascore'].unique()

array([nan, 96, 84, 80, 76, 70, 'NA', 69, 81, 75, 72, 66, 67, 77, 91, 89,
       83, 61, 88, 65, 94, 57, 86, 87, 92, 79, 82, 58, 74, 85, 90, 68, 71,
       60, 73, 59, 64, 54, 53, 78, 51, 44, 63, 38, 56, 49, 52, 62, 93, 48,
       34, 95, 43, 55, 24, 46, 41, 20, 39, 45, 35, 47, 40, 36, 50, 32, 37,
       33, 42, 27, 29, 30], dtype=object)

In [13]:
df_filtrado['metascore'] = df_filtrado['metascore'].replace('NA', None)
df_filtrado.dropna(subset=['metascore'], inplace=True)

Descartamos los valores nulos que quedan en generos

In [14]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2425 entries, 28 to 32117
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        2419 non-null   object 
 1   release_date  2425 non-null   object 
 2   tags          2424 non-null   object 
 3   specs         2422 non-null   object 
 4   price         2425 non-null   float64
 5   sentiment     2425 non-null   object 
 6   metascore     2425 non-null   object 
dtypes: float64(1), object(6)
memory usage: 151.6+ KB


In [15]:
df_filtrado.dropna(subset=['genres'], inplace=True)
df_filtrado.dropna(subset=['specs'], inplace=True)

Insertamos el df ya transformado en un archivo CSV listo para el EDA

In [16]:
df_filtrado.to_csv('steam_games_ETL.csv', index=False)